In [57]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [58]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [59]:
BATCH_SIZE = 2
LEARNING_RATE = 0.01
EPOCHS = 50

In [60]:
def prep_data(input_data):
    return DataLoader(dataset=input_data, batch_size=BATCH_SIZE, shuffle=True)

In [61]:
def test_loop(data, model, loss_function):
    size = len(data.dataset)
    num_batches = len(data)
    

In [62]:
def seperate_variables(dataset, target_value):
    for datapoint in dataset:
        X = datapoint.drop(target_value, axis=1)
        y = datapoint[target_value]
        return (X, y)
        

In [63]:
csv_filepath = "./final_harvest_data.csv"

data = pd.read_csv(csv_filepath)
data = data.drop(['date', 'index', 'plant_id', 'tray_id', 'row', 'column'], axis=1)

data


,LFW_g,LDW_g,LA_mm2,length_mm,width_mm,height_mm,plant_area,plant_convex_hull_area,plant_solidity,plant_perimeter,plant_width,plant_height,plant_longest_path,plant_convex_hull_vertices,plant_ellipse_major_axis,plant_ellipse_minor_axis,plant_ellipse_angle,plant_ellipse_eccentricity
0,1.30,0.078,31.95,4.3,5.2,5.8,1211,1456.0,0.831731,189.923880,49,42,346,16,45.684494,40.965988,92.878510,0.442608
1,2.10,0.148,44.10,5.3,5.7,5.5,1412,1556.5,0.907164,181.338094,56,41,370,20,53.368065,37.484673,96.255425,0.711802
2,3.36,0.196,67.61,7.3,6.5,8.9,1303,1766.5,0.737617,247.865005,49,54,333,19,46.356819,43.059174,2.102176,0.370421
3,3.07,0.184,66.98,5.8,7.6,7.5,1601,1787.0,0.895915,203.480229,44,59,395,21,55.633369,38.548523,15.127802,0.721031
4,3.17,0.187,68.74,7.2,8.0,7.1,1919,2372.0,0.809022,263.421354,62,58,454,19,60.012882,48.875011,55.774792,0.580292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,57.96,3.010,726.46,18.5,18.5,13.0,19084,22974.0,0.830678,767.938160,193,167,1367,33,190.580887,137.733093,62.138748,0.691160
161,81.46,3.880,1001.79,21.0,21.5,14.8,18373,21556.5,0.852318,661.612260,164,189,1317,29,176.040924,141.141724,22.056726,0.597653
162,140.84,6.380,1707.14,23.0,22.5,16.9,23374,26050.0,0.897274,683.754395,186,194,1427,31,191.379974,164.496170,175.959305,0.511091
163,108.17,5.250,1364.18,31.4,20.5,16.6,23457,25678.5,0.913488,671.754395,198,179,1364,31,194.815384,159.870819,122.656914,0.571464


In [64]:
LFW_X = data.drop('LFW_g', axis=1)
LFW_y = data['LFW_g']
LDW_X = data.drop('LDW_g', axis=1)
LDW_y = data['LDW_g']
LA_X = data.drop('plant_area', axis=1)
LA_y = data['plant_area']

LFW_X

,LDW_g,LA_mm2,length_mm,width_mm,height_mm,plant_area,plant_convex_hull_area,plant_solidity,plant_perimeter,plant_width,plant_height,plant_longest_path,plant_convex_hull_vertices,plant_ellipse_major_axis,plant_ellipse_minor_axis,plant_ellipse_angle,plant_ellipse_eccentricity
0,0.078,31.95,4.3,5.2,5.8,1211,1456.0,0.831731,189.923880,49,42,346,16,45.684494,40.965988,92.878510,0.442608
1,0.148,44.10,5.3,5.7,5.5,1412,1556.5,0.907164,181.338094,56,41,370,20,53.368065,37.484673,96.255425,0.711802
2,0.196,67.61,7.3,6.5,8.9,1303,1766.5,0.737617,247.865005,49,54,333,19,46.356819,43.059174,2.102176,0.370421
3,0.184,66.98,5.8,7.6,7.5,1601,1787.0,0.895915,203.480229,44,59,395,21,55.633369,38.548523,15.127802,0.721031
4,0.187,68.74,7.2,8.0,7.1,1919,2372.0,0.809022,263.421354,62,58,454,19,60.012882,48.875011,55.774792,0.580292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,3.010,726.46,18.5,18.5,13.0,19084,22974.0,0.830678,767.938160,193,167,1367,33,190.580887,137.733093,62.138748,0.691160
161,3.880,1001.79,21.0,21.5,14.8,18373,21556.5,0.852318,661.612260,164,189,1317,29,176.040924,141.141724,22.056726,0.597653
162,6.380,1707.14,23.0,22.5,16.9,23374,26050.0,0.897274,683.754395,186,194,1427,31,191.379974,164.496170,175.959305,0.511091
163,5.250,1364.18,31.4,20.5,16.6,23457,25678.5,0.913488,671.754395,198,179,1364,31,194.815384,159.870819,122.656914,0.571464


In [65]:
LFW_X_train, LFW_X_test, LFW_y_train, LFW_y_test = train_test_split(LFW_X, LFW_y, test_size=0.2, random_state=42)
LDW_X_train, LDW_X_test, LDW_y_train, LDW_y_test = train_test_split(LDW_X, LDW_y, test_size=0.2, random_state=42)
LA_X_train, LA_X_test, LA_y_train, LA_y_test = train_test_split(LA_X, LA_y, test_size=0.2, random_state=42)

LFW_X_train

,LDW_g,LA_mm2,length_mm,width_mm,height_mm,plant_area,plant_convex_hull_area,plant_solidity,plant_perimeter,plant_width,plant_height,plant_longest_path,plant_convex_hull_vertices,plant_ellipse_major_axis,plant_ellipse_minor_axis,plant_ellipse_angle,plant_ellipse_eccentricity
84,3.6200,902.27,19.0,20.3,14.7,23787,26418.5,0.900392,727.269114,198,184,1350,34,192.520447,162.562088,125.322227,0.535731
2,0.1960,67.61,7.3,6.5,8.9,1303,1766.5,0.737617,247.865005,49,54,333,19,46.356819,43.059174,2.102176,0.370421
94,6.3100,1615.69,24.3,25.3,17.7,33899,38796.0,0.873776,1176.903690,253,211,1771,29,248.432251,187.253662,75.101181,0.657171
45,0.5723,146.31,9.6,9.9,9.3,4184,5088.0,0.822327,337.320849,93,88,624,24,82.943108,79.204681,104.443810,0.296838
42,0.9157,214.12,12.1,12.3,10.1,5435,7078.5,0.767818,453.688380,100,103,752,22,95.243637,87.623962,133.338226,0.391923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,0.5414,131.13,11.1,11.5,9.5,4098,5005.0,0.818781,385.788885,87,77,633,27,89.748093,68.229294,109.227425,0.649653
106,3.1800,763.37,19.5,19.0,14.2,20328,22208.5,0.915325,675.553384,201,170,1349,34,192.158005,138.995163,60.211781,0.690494
14,0.1100,35.17,6.8,5.7,3.8,1585,1694.5,0.935379,171.095453,49,46,354,20,50.709599,41.330761,123.980499,0.579393
92,7.7900,1968.38,25.0,26.5,18.5,27935,33339.5,0.837895,844.322936,206,215,1472,36,208.834915,185.417541,22.582560,0.460101


In [66]:
LFW_X_train = LFW_X_train.to_numpy().astype(np.double)
LFW_X_test = LFW_X_test.to_numpy()
LFW_y_train = LFW_y_train.to_numpy()
LFW_y_test = LFW_y_test.to_numpy()
LDW_X_train = LDW_X_train.to_numpy()
LDW_X_test = LDW_X_test.to_numpy()
LDW_y_train = LDW_y_train.to_numpy()
LDW_y_test = LDW_y_test.to_numpy()
LA_X_train = LA_X_train.to_numpy()
LA_X_test = LA_X_test.to_numpy()
LA_y_train = LA_y_train.to_numpy()
LA_y_test = LA_y_test.to_numpy()

LFW_X_train

array([[3.62000000e+00, 9.02270000e+02, 1.90000000e+01, ...,
        1.62562088e+02, 1.25322227e+02, 5.35731032e-01],
       [1.96000000e-01, 6.76100000e+01, 7.30000000e+00, ...,
        4.30591736e+01, 2.10217595e+00, 3.70421308e-01],
       [6.31000000e+00, 1.61569000e+03, 2.43000000e+01, ...,
        1.87253662e+02, 7.51011810e+01, 6.57171185e-01],
       ...,
       [1.10000000e-01, 3.51700000e+01, 6.80000000e+00, ...,
        4.13307610e+01, 1.23980499e+02, 5.79393326e-01],
       [7.79000000e+00, 1.96838000e+03, 2.50000000e+01, ...,
        1.85417541e+02, 2.25825596e+01, 4.60101010e-01],
       [5.13000000e+00, 1.35188000e+03, 2.20000000e+01, ...,
        1.61043579e+02, 3.57503295e+00, 3.98170560e-01]])

In [67]:
LFW_X_train = torch.from_numpy(LFW_X_train)
LFW_X_test = torch.from_numpy(LFW_X_test)
LFW_y_train = torch.from_numpy(LFW_y_train)
LFW_y_test = torch.from_numpy(LFW_y_test)
LDW_X_train = torch.from_numpy(LDW_X_train)
LDW_X_test = torch.from_numpy(LDW_X_test)
LDW_y_train = torch.from_numpy(LDW_y_train)
LDW_y_test = torch.from_numpy(LDW_y_test)
LA_X_train = torch.from_numpy(LA_X_train)
LA_X_test = torch.from_numpy(LA_X_test)
LA_y_train = torch.from_numpy(LA_y_train)
LA_y_test = torch.from_numpy(LA_y_test)

In [68]:
LFW_X_train = prep_data(LFW_X_train)
LFW_X_test = prep_data(LFW_X_test)
LFW_y_train = prep_data(LFW_y_train)
LFW_y_test = prep_data(LFW_y_test)
LDW_X_train = prep_data(LDW_X_train)
LDW_X_test = prep_data(LDW_X_test)
LDW_y_train = prep_data(LDW_y_train)
LDW_y_test = prep_data(LDW_y_test)
LA_X_train = prep_data(LA_X_train)
LA_X_test = prep_data(LA_X_test)
LA_y_train = prep_data(LA_y_train)
LA_y_test = prep_data(LA_y_test)


In [69]:
class DeepNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc1 = nn.Linear(17,16)
        self.fc2 = nn.Linear(16,1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x.double()
        



In [70]:
def train_loop(dataset, model, loss_function, optimizer):
    
    for point in dataset:

        X, y = seperate_variables(dataset, "LFW_g")

        model.zero_grad()

        prediction = model(X)
        loss = loss_function(prediction, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f"loss {loss}")

In [71]:
model = DeepNeuralNetwork()
model = model.double()
print(model)

DeepNeuralNetwork(
  (fc1): Linear(in_features=17, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=1, bias=True)
)


In [72]:
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

for t in range(EPOCHS):
    print(f"Epoch {t+1}\n------------------------------------")
    train_loop(LFW_X_train, model, loss_function, optimizer)
print("Training complete!")

Epoch 1
------------------------------------


AttributeError: 'Tensor' object has no attribute 'drop'